In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [2]:
class ChestXrayDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Image
        img_path = os.path.join(self.image_dir, row["image_id"])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # Clinical / X-ray parameters
        clinical_features = torch.tensor(
            [
                row["age"],
                row["gender"],
                row["exposure"],
                row["rotation"]
            ],
            dtype=torch.float32
        )

        # Label
        label = torch.tensor(row["label"], dtype=torch.long)

        return image, clinical_features, label


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [5]:
train_dataset = ChestXrayDataset(
    csv_file=r"C:\Users\sureb\my_data\primary_data.csv",
    image_dir=r"C:\Users\sureb\my_data\images\primary_data",
    transform=transform
)

test_dataset = ChestXrayDataset(
    csv_file=r"C:\Users\sureb\my_data\secondary_data.csv",
    image_dir=r"C:\Users\sureb\my_data\images\secondary_data",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [6]:
class ImageCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(128 * 28 * 28, 256)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [7]:
class ClinicalNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(4, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)


In [8]:
class FusedModel(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()

        self.image_net = ImageCNN()
        self.clinical_net = ClinicalNet()

        self.classifier = nn.Sequential(
            nn.Linear(256 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, clinical_data):
        img_features = self.image_net(image)
        clinical_features = self.clinical_net(clinical_data)

        fused = torch.cat((img_features, clinical_features), dim=1)
        output = self.classifier(fused)

        return output


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FusedModel(num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [10]:
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, clinical_data, labels in loader:
            images = images.to(device)
            clinical_data = clinical_data.to(device)
            labels = labels.to(device)

            outputs = model(images, clinical_data)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(loader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy


In [13]:
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images, clinical_data, labels in train_loader:
        images = images.to(device)
        clinical_data = clinical_data.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images, clinical_data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    _,train_acc= evaluate(model, train_loader)
    test_loss, test_acc = evaluate(model, test_loader)

    print(
        f"Epoch [{epoch+1}/{epochs}] | "
        f"Train Loss: {train_loss:.4f} | "
        f"Train Acc: {train_acc:.2f}% |"
        f"Test Loss: {test_loss:.4f} | "
        f"Test Acc: {test_acc:.2f}%"
    )


Epoch [1/5] | Train Loss: 0.0633 | Train Acc: 98.10% |Test Loss: 1.8528 | Test Acc: 42.08%
Epoch [2/5] | Train Loss: 0.0815 | Train Acc: 99.05% |Test Loss: 3.0938 | Test Acc: 30.77%
Epoch [3/5] | Train Loss: 0.1086 | Train Acc: 99.52% |Test Loss: 2.3661 | Test Acc: 33.03%
Epoch [4/5] | Train Loss: 0.0504 | Train Acc: 100.00% |Test Loss: 2.2164 | Test Acc: 30.77%
Epoch [5/5] | Train Loss: 0.0636 | Train Acc: 99.52% |Test Loss: 1.9049 | Test Acc: 42.99%


In [14]:
test_loss, test_acc = evaluate(model, test_loader)

print(f"Final Test Accuracy: {test_acc:.2f}%")


Final Test Accuracy: 42.99%
